In [295]:
import sys 
import os
from itertools import groupby
from math import floor
print(os. getcwd() )

/Users/maosheng/Documents/scholp_dataset


In [296]:

def generate_simplices(top_dim=7, file_name='email-Enron',unique_simplices=False,percentage_time=1):
    '''
    top_dim: the largest simplicial order, i.e., the simplicial complex order
    file_name: 
    unique_simplices: if to generate the unique simplices, since this dataset contains the same simplex for many times over a certain length of time stamps
        - 'True': we also include the duration of the time stamps of each simplex in the dict value
        - 'False': generate simplices allowed to be duplicate        
    '''
    file_path = './ScHoLP-Data-1.0/'+file_name
    with open(file_path+'/'+file_name+'-nverts.txt') as f:
        simplex_dim = [line.rstrip('\n') for line in f]    
    with open(file_path+'/'+file_name+'-simplices.txt') as f:
        simplex_vertices = [line.rstrip('\n') for line in f]
        
    simplex_dim = simplex_dim[:floor(percentage_time*len(simplex_dim))]
    start=0
    simplex_vertices_tupled = []
    for count in simplex_dim:
        end = start + int(count) 
        sublist = simplex_vertices[start:end]
        sublist = [int(x) for x in sublist]
        simplex_vertices_tupled.append(sublist)
        start=end
    
    list_simplices = [None]*top_dim
    for k in range(top_dim):
        list_simplices[k] = [sorted(id_vertices) for id_simp,id_vertices in enumerate(simplex_vertices_tupled) if len(id_vertices) == k+1]
        list_simplices[k].sort() 
        freq_simplices = [len(list(group)) for key, group in groupby(list_simplices[k])]
        if unique_simplices:
            list_simplices[k]=[tuple(list(x)) for x in set(tuple(x) for x in list_simplices[k])]
            list_simplices[k].sort() 
            '''list_simplices[k]:{'key':idx,{'value1':[v0,v1,...,v_k]},'value2':x or w}'''
            # list_simplices[k] = dict(zip(range(len(list_simplices[k])),list_simplices[k]))
            list_simplices[k] = dict(zip(range(len(list_simplices[k])),zip(list_simplices[k],freq_simplices)))
            # weights_simplices[k] = dict(zip(range(len(list_simplices[k])),freq_simplices))
        else:
            list_simplices[k]=[tuple(x) for x in list_simplices[k]]   
            list_simplices[k].sort()       
            list_simplices[k] = dict(zip(range(len(list_simplices[k])),list_simplices[k]))
    if unique_simplices:
        return list_simplices
    else:
        return list_simplices

In [297]:
file_name = 'email-Enron'
list_simplices= generate_simplices(top_dim=4,file_name=file_name,unique_simplices=True,percentage_time=1)

In [298]:
print((list_simplices[1]))
print(list_simplices[2])



{0: ((1, 2), 2), 1: ((1, 4), 4), 2: ((1, 8), 1), 3: ((1, 20), 2), 4: ((1, 23), 2), 5: ((1, 29), 4), 6: ((1, 41), 21), 7: ((1, 51), 23), 8: ((1, 52), 2), 9: ((1, 57), 1), 10: ((1, 62), 3), 11: ((1, 63), 22), 12: ((1, 65), 11), 13: ((1, 71), 2), 14: ((1, 79), 1), 15: ((1, 85), 6), 16: ((1, 97), 1), 17: ((1, 98), 4), 18: ((1, 107), 3), 19: ((1, 112), 2), 20: ((1, 117), 6), 21: ((1, 120), 4), 22: ((1, 122), 1), 23: ((1, 125), 3), 24: ((1, 129), 2), 25: ((1, 132), 1), 26: ((1, 133), 1), 27: ((1, 136), 2), 28: ((1, 142), 1), 29: ((1, 147), 2), 30: ((2, 11), 1), 31: ((2, 40), 8), 32: ((2, 41), 2), 33: ((2, 55), 1), 34: ((2, 56), 4), 35: ((2, 58), 2), 36: ((2, 61), 5), 37: ((2, 63), 113), 38: ((2, 72), 45), 39: ((2, 79), 1), 40: ((2, 80), 20), 41: ((2, 85), 1), 42: ((2, 93), 2), 43: ((2, 95), 55), 44: ((2, 99), 2), 45: ((2, 111), 2), 46: ((2, 114), 4), 47: ((2, 115), 15), 48: ((2, 117), 1), 49: ((2, 128), 2), 50: ((2, 129), 3), 51: ((2, 137), 36), 52: ((2, 140), 13), 53: ((2, 147), 32), 54: ((

In [299]:
''' 
apparently in this dataset, the simplices are not strictly a simplex
some open trianlges are in the simplices list as well
we can then determine if a simplex is positive (closed) or negative (open)
'''
import itertools
def list_faces(list_simplices):
    list_faces = list_simplices
    for k in range(2,len(list_simplices)):
        for idx,idx_vertices in list_simplices[k].items():
            face_vertices = (list(itertools.combinations(idx_vertices[0],k)))
            # print(face_vertices)
            face_idx = ([face_id for face_id,face_vertex in list_simplices[k-1].items() if face_vertex[0] in face_vertices])
            # print(face_idx)
            list_faces[k][idx] = (tuple(face_idx),list_faces[k][idx][1])

    return list_faces

list_faces = list_faces(list_simplices)
print(list_faces[0])
print(list_faces[1])

print(list_faces[2])


{0: ((2,), 12), 1: ((3,), 1), 2: ((6,), 1), 3: ((8,), 3), 4: ((12,), 7), 5: ((13,), 7), 6: ((15,), 4), 7: ((17,), 2), 8: ((19,), 5), 9: ((20,), 45), 10: ((21,), 14), 11: ((23,), 1), 12: ((29,), 2), 13: ((30,), 2), 14: ((33,), 3), 15: ((37,), 54), 16: ((41,), 1), 17: ((42,), 1), 18: ((43,), 1), 19: ((44,), 4), 20: ((46,), 5), 21: ((47,), 1), 22: ((55,), 1), 23: ((56,), 28), 24: ((57,), 4), 25: ((61,), 15), 26: ((62,), 1), 27: ((63,), 3), 28: ((65,), 7), 29: ((68,), 2), 30: ((70,), 1), 31: ((73,), 12), 32: ((74,), 7), 33: ((77,), 3), 34: ((83,), 2), 35: ((86,), 5), 36: ((90,), 13), 37: ((95,), 2), 38: ((96,), 4), 39: ((102,), 1), 40: ((104,), 6), 41: ((108,), 32), 42: ((110,), 13), 43: ((111,), 9), 44: ((112,), 7), 45: ((114,), 1), 46: ((115,), 1), 47: ((120,), 1), 48: ((131,), 1), 49: ((132,), 24), 50: ((134,), 1), 51: ((139,), 3), 52: ((143,), 15), 53: ((144,), 28), 54: ((146,), 2)}
{0: ((1, 2), 2), 1: ((1, 4), 4), 2: ((1, 8), 1), 3: ((1, 20), 2), 4: ((1, 23), 2), 5: ((1, 29), 4), 6: (